In [1]:
########## Import Libararies
import os
import pickle
import glob
from tqdm.notebook import tqdm

import numpy as np
from skimage import measure
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

import open3d as o3d
from compas.geometry import trimesh_remesh
from compas.datastructures import Mesh
import nibabel as nib

# from util import *

import csv
import copy
import pickle
import random


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [31]:
random.seed(42) 
def sub_sort(folder_path):
    file_groups = {}
    
    for filename in os.listdir(folder_path):

        if filename.endswith('.nii.gz'):
            filename = filename[:-7] # .nii.gz elimination
        elif filename.endswith('.npy_L') or filename.endswith('.npy_R'):
            filename = filename[:-6] # .npy_L elimination
        elif filename.endswith('.nii.gz_R') or filename.endswith('.nii.gz_L'):
            filename = filename[:-9] # .npy_L elimination
        elif filename.endswith('_R') or filename.endswith('_L'):
            filename = filename[:-2] # .npy_L eliminations

        # print(filename)
        parts = filename.split('_')
        file_id = '_'.join(map(str, parts[:-5]))
        # print(file_id)
        days = (float(parts[-3]))
        
        # making group by id
        if file_id not in file_groups:
            file_groups[file_id] = []
        
        # filename & days as tuple
        file_groups[file_id].append((filename, days))

    # grouping with same id and sorting by days
    sorted_file_groups = []
    for file_id, files in file_groups.items():
        files_sorted = sorted(files, key=lambda x: x[1])  # x[1]이 days 값
        sorted_file_groups.append([file[0] for file in files_sorted])

    return sorted_file_groups

def get_sub_list(folder_path, ncad = 0, agetype = 60, LR= 'L'):
    files = sub_sort(folder_path)
    file_list, age_list = [], []
    for file in files:
        f = file[0]
        split = f.split("\\")[-1].split("_")
        if (LR == 'L' and not split[0].startswith("R")) or (LR == 'R' and split[0].startswith("R")):
        # if not split[0].startswith("R"):
            ncad_flag = split[-2]
            age = split[2]
            if int(ncad_flag) == ncad:
                # print(f)
                npy_file = glob.glob(rf"{folder_path}\{f}*\*.npy")
                if npy_file!=[]:
                    loss = float(npy_file[0].split("_")[-1].split(".")[0])
                    # print(loss)
                    if loss <10:
                        if (ncad==0 and float(age)>59 and float(age)<90) :
                            if ncad==0 and float(age)<70 and agetype==60:
                                
                                if random.random()>0.5:
                            
                                    file_list.append(npy_file[0])
                                    age_list.append(float(age))
                            elif ncad==0 and float(age)>=80 and float(age)<90 and agetype==80:
                                file_list.append(npy_file[0])
                                age_list.append(float(age))
    # print(file_list)
    return file_list, age_list


import open3d as o3d
import csv
import glob
import numpy as np
import pickle
import os


tempdata = pickle.load(open(rf"D:\IdeaProjects\longi_node\whole_brain_structure\temp_meshes\cuttail_temp.pkl","rb"))
lvmesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata['lv']))

nc_list, nc_age_list = [], []
ad_list, ad_age_list = [], []
for dataset in ['oasis2', 'oasis3', 'adni', 'aibl']:
    folder = rf"D:\WJ\{dataset}_lv"
    nc, ncage = get_sub_list(folder, agetype=60)
    ad, adage = get_sub_list(folder, agetype=80)
    nc_list += nc
    nc_age_list += ncage
    ad_list += ad
    ad_age_list += adage
nc_age_list = np.asarray(nc_age_list)
ad_age_list = np.asarray(ad_age_list)

nc_list_R, nc_age_list_R = [], []
ad_list_R, ad_age_list_R = [], []
for dataset in ['oasis2', 'oasis3', 'adni', 'aibl']:
    folder = rf"D:\WJ\{dataset}_lv"
    nc, ncage = get_sub_list(folder, agetype=60, LR='R')
    ad, adage = get_sub_list(folder, agetype=80, LR='R')
    nc_list_R += nc
    nc_age_list_R += ncage
    ad_list_R += ad
    ad_age_list_R += adage
nc_age_list_R = np.asarray(nc_age_list_R)
ad_age_list_R = np.asarray(ad_age_list_R)


print(nc_age_list_R.shape, ad_age_list_R.shape)

print("NC: mean, std, min, max, sub# : ", np.mean(nc_age_list),np.std(nc_age_list),  np.min(ad_age_list), np.max(nc_age_list), np.shape(nc_age_list))
print("AD: mean, std, min, max, sub# : ", np.mean(ad_age_list),np.std(ad_age_list), np.min(ad_age_list), np.max(ad_age_list), np.shape(ad_age_list))

print("NC: mean, std, min, max, sub# : ", np.mean(nc_age_list_R),np.std(nc_age_list_R),  np.min(ad_age_list_R), np.max(nc_age_list_R), np.shape(nc_age_list_R))
print("AD: mean, std, min, max, sub# : ", np.mean(ad_age_list_R),np.std(ad_age_list_R), np.min(ad_age_list_R), np.max(ad_age_list_R), np.shape(ad_age_list_R))

(189,) (95,)
NC: mean, std, min, max, sub# :  65.45755813953488 2.8289205912743602 80.0 69.6 (172,)
AD: mean, std, min, max, sub# :  83.52345679012346 2.8204400149201083 80.0 89.2 (81,)
NC: mean, std, min, max, sub# :  65.63280423280422 2.966529882486571 80.0 69.9 (189,)
AD: mean, std, min, max, sub# :  83.48210526315789 2.8442362381478357 80.0 89.2 (95,)


In [32]:
# a=np.asarray([2,3,4,5])
# b=np.asarray([0,1,1,0])
# # remain a where b is 1
# a = a[b==1]
# print(a)
id_list = []
nc_list = np.asarray(nc_list)
ad_list = np.asarray(ad_list)
nc_list_R = np.asarray(nc_list_R)
ad_list_R = np.asarray(ad_list_R)
for ind,li in enumerate([nc_list, ad_list, nc_list_R, ad_list_R]):
    ids = []
    for f in li:
        split = f.split("\\")[-2].split("_")[0]
        if ind<2:
            
            
            ids.append(split)
        else:
            ids.append(split[1:])
    id_list.append(ids)

print(id_list)
mask_nc_list = np.asarray([int(x in id_list[2]) for x in id_list[0]])
mask_ad_list = np.asarray([int(x in id_list[3]) for x in id_list[1]])

mask_nc_list_R = np.asarray([int(x in id_list[0]) for x in id_list[2]])
mask_ad_list_R = np.asarray([int(x in id_list[1]) for x in id_list[3]])

print(mask_nc_list)

nc_list = nc_list[mask_nc_list==1]
ad_list = ad_list[mask_ad_list==1]
nc_list_R = nc_list_R[mask_nc_list_R==1]
ad_list_R = ad_list_R[mask_ad_list_R==1]
    
    

[['OAS20022', 'OAS20049', 'OAS20067', 'OAS20076', 'OAS20145', 'OAS20186', 'OAS30001', 'OAS30003', 'OAS30006', 'OAS30015', 'OAS30034', 'OAS30042', 'OAS30080', 'OAS30083', 'OAS30086', 'OAS30092', 'OAS30096', 'OAS30099', 'OAS30103', 'OAS30107', 'OAS30115', 'OAS30126', 'OAS30127', 'OAS30137', 'OAS30178', 'OAS30204', 'OAS30216', 'OAS30220', 'OAS30242', 'OAS30257', 'OAS30268', 'OAS30269', 'OAS30275', 'OAS30288', 'OAS30291', 'OAS30353', 'OAS30367', 'OAS30372', 'OAS30402', 'OAS30454', 'OAS30456', 'OAS30458', 'OAS30465', 'OAS30484', 'OAS30513', 'OAS30555', 'OAS30570', 'OAS30594', 'OAS30615', 'OAS30620', 'OAS30625', 'OAS30626', 'OAS30631', 'OAS30642', 'OAS30663', 'OAS30664', 'OAS30671', 'OAS30677', 'OAS30681', 'OAS30688', 'OAS30690', 'OAS30694', 'OAS30710', 'OAS30722', 'OAS30723', 'OAS30742', 'OAS30750', 'OAS30754', 'OAS30764', 'OAS30765', 'OAS30792', 'OAS30794', 'OAS30807', 'OAS30820', 'OAS30844', 'OAS30854', 'OAS30876', 'OAS30881', 'OAS30920', 'OAS30962', 'OAS30979', 'OAS30992', 'OAS31005', 'O

# Generate Average Mesh

In [ ]:
# nc_age_list = nc_age_list[mask_nc_list==1]
# ad_age_list = ad_age_list[mask_ad_list==1]
# nc_age_list_R = nc_age_list_R[mask_nc_list_R==1]
# ad_age_list_R = ad_age_list_R[mask_ad_list_R==1]

print("NC: mean, std, min, max, sub# : ", np.mean(nc_age_list),np.std(nc_age_list),  np.min(ad_age_list), np.max(nc_age_list), np.shape(nc_age_list))
print("AD: mean, std, min, max, sub# : ", np.mean(ad_age_list),np.std(ad_age_list), np.min(ad_age_list), np.max(ad_age_list), np.shape(ad_age_list))

print("NC: mean, std, min, max, sub# : ", np.mean(nc_age_list_R),np.std(nc_age_list_R),  np.min(ad_age_list_R), np.max(nc_age_list_R), np.shape(nc_age_list_R))
print("AD: mean, std, min, max, sub# : ", np.mean(ad_age_list_R),np.std(ad_age_list_R), np.min(ad_age_list_R), np.max(ad_age_list_R), np.shape(ad_age_list_R))

nc_list = nc_list.tolist()
ad_list = ad_list.tolist()
nc_list_R = nc_list_R.tolist()
ad_list_R = ad_list_R.tolist()


NC: mean, std, min, max, sub# :  65.2175 3.03483010232863 80.0 69.6 (80,)
AD: mean, std, min, max, sub# :  83.51750000000001 2.837506607921822 80.0 89.2 (80,)
NC: mean, std, min, max, sub# :  65.2175 3.03483010232863 80.0 69.6 (80,)
AD: mean, std, min, max, sub# :  83.51750000000001 2.837506607921822 80.0 89.2 (80,)


In [36]:
modes = ["0627"] #"manual_l", "manual_r","synthseg_l","synthseg_r","synthseg_l_new","synthseg_r_new","synthseg_l_new_2001","synthseg_r_new_2001"]
exclude_list = []

temp_mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata['lv']))
# output_path = rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\edinburgh\edinburgh\0124\{mode}\out"
# filelist = glob.glob(rf"{l_save_path}\*.pkl")
# ######### Read Hippo Temp Mesh #########

temp_vert_num = np.asarray(temp_mesh.vertices).shape[0]


# temp_mesh.compute_vertex_normals()
# temp_mesh.paint_uniform_color([0.7, 1.0, 0.7])
count=0
# print(filelist)
index= 0
i_tag=0
filelist = nc_list + ad_list
new_verts = np.empty([len(filelist), temp_vert_num, 3])
for j, key in tqdm(enumerate(filelist)):
    sub = key.split("\\")[-1].split("_")[0]

    with open(key, 'rb') as f:
        Temp_saved = copy.deepcopy(temp_mesh)
        # pcd = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(points))
        try:
            # new_vert_file = glob.glob(rf"{output_path}\{sub}\5000*.npy")
            new_vert = np.load(f)[0]
            new_verts[index, :, :] = new_vert
            draw_mesh = copy.deepcopy(temp_mesh)
            draw_mesh.vertices = o3d.utility.Vector3dVector(new_vert)
            draw_mesh.compute_vertex_normals()
            draw_mesh.paint_uniform_color([0.7, 0.7, 1.0])
            index+=1
            # o3d.visualization.draw_geometries([draw_mesh,pcd], mesh_show_wireframe=True)
            # if index>10: break
        except:
            print(sub)

# print(count)
mean_vertices = np.mean(new_verts, axis=0)

avg_mesh = copy.deepcopy(temp_mesh)
if i_tag==0: avg_mesh.vertices = o3d.utility.Vector3dVector(mean_vertices)
mean_vertices =np.asarray(avg_mesh.vertices)
avg_mesh.compute_vertex_normals()
avg_mesh.paint_uniform_color([0.5, 0.5, 1.0])
o3d.visualization.draw_geometries([temp_mesh, avg_mesh], mesh_show_wireframe=True)

colors = np.empty((len(filelist), temp_vert_num, 3))
diffs = np.empty((len(filelist), temp_vert_num, 3))
for i in range(len(filelist)):#len(past_excel.keys())):
    sample = new_verts[i, :, :]
    diffs[i,:,:] =sample - mean_vertices
    # print(np.unique(diffs[i,:,:]))
    colors[i, :, 0] = np.linalg.norm(sample - mean_vertices, axis=1)
    colors[i, :, 1] = np.linalg.norm(sample - mean_vertices, axis=1)
    colors[i, :, 2] = np.linalg.norm(sample - mean_vertices, axis=1)
    # if colors[i, :, 0].mean()>10: print(i) 
var_color = np.var(colors, axis=0)/20


avg_mesh.vertex_colors = o3d.utility.Vector3dVector(var_color)

o3d.visualization.draw_geometries([avg_mesh], mesh_show_wireframe=True)
import matplotlib.pyplot as plt
import numpy as np

# Normalize your value (if not already between 0 and 1)
normalized_value = var_color[:,0]  # Example value, should be between 0 and 1
# print(np.var(colors))

# Get the viridis colormap
viridis = plt.cm.viridis

# Map the normalized value to a color
color = viridis(normalized_value) # Euclidean distance from mean
# print(color.shape)

avg_mesh.vertex_colors = o3d.utility.Vector3dVector(color[:,:3])

# if i_tag==0:o3d.io.write_triangle_mesh(rf"0124_deformity_result/fs-{mode}_avg.obj", avg_mesh)
o3d.visualization.draw_geometries([avg_mesh], mesh_show_wireframe=True)

import numpy as np

# For debugging purposes
# print(np.unique(diffs[0,:,:]))

avg_mesh_normals = np.asarray(avg_mesh.vertex_normals)
viridis = plt.cm.viridis
disps = np.empty((len(filelist), temp_vert_num))
for index, file in tqdm(enumerate(filelist)):
    sub_id = file.split("\\")[-1].split("_")[0]
    # print(colors[index,:,0], sub_id)
    dot_products = np.einsum('ij,ij->i', avg_mesh_normals, diffs[index])
    dot_products[dot_products<0] = -1
    dot_products[dot_products>=0] = 1
    disp = dot_products*colors[index,:,0] # signed Euclidean distance from mean
    #print(disp.shape)
    disps[index]=disp
np.save(f"ncad_L_age.npy", disps)
        



0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
nc_ad_disps = np.load(f"ncad_.npy")
print(nc_ad_disps.shape)

(478, 2826)


In [37]:
data_L = np.load("ncad_L_age.npy")
data_R = np.load("ncad_R_age.npy")

# print(data_L.shape, data_R.shape)

In [39]:
from scipy.stats import levene, bartlett, fligner, ttest_ind, shapiro, brunnermunzel
from scipy.stats import mannwhitneyu
from scipy import stats
nc_ad_disps = data_R

shapiro1 = []
shapiro2 = []
levene12 = []
munzel = []
p_value = np.zeros(nc_ad_disps.shape[1])
p_value2 = np.zeros(nc_ad_disps.shape[1])
for i in range(nc_ad_disps.shape[1]):
    data1 = nc_ad_disps[:len(nc_list),i]
    data2 = nc_ad_disps[len(nc_list):,i]
    stat_shapiro1, p_shapiro1 = shapiro(data1)
    stat_shapiro2, p_shapiro2 = shapiro(data2)
    stat_levene1, p_levene = levene(data1, data2)
    shapiro1.append(p_shapiro1)
    shapiro2.append(p_shapiro2)
    levene12.append(p_levene)
    stat, p1 = brunnermunzel(data1, data2)
    stat, p2 = mannwhitneyu(data1, data2, alternative='two-sided')
    munzel.append(p1)
    p_value[i] = p1
    p_value2[i] = p2
print(shapiro1)
print(np.sum(np.asarray(shapiro1)>0.05), np.sum(np.asarray(shapiro2)>0.05)) # 정규성
print(np.sum(np.asarray(levene12)>0.05)) # 등분산성
print(np.sum(np.asarray(munzel)<=0.05))
    

    
viridis = plt.cm.jet_r
color = viridis(p_value2*10)
avg_mesh.vertex_colors = o3d.utility.Vector3dVector(color[:,:3])
o3d.io.write_triangle_mesh(rf"age_mannwhitneyu_R.obj", avg_mesh)
np.save(f"age_p_R.npy", p_value2)
o3d.visualization.draw_geometries([avg_mesh], mesh_show_wireframe=True)




[0.06206279799928097, 0.06676480683265577, 0.05458340027697364, 0.01386906596655364, 0.11634741463028347, 0.10489115429420742, 0.1184001752477411, 0.6141344830921469, 0.06034382720720602, 0.0705356079558531, 0.40474156491669483, 0.011389786191118271, 0.059722781347021386, 0.150576201412621, 0.02113702103174508, 0.04114457491070611, 0.013316264014936907, 0.017966891723239748, 0.09078511697315114, 0.06097686537242209, 0.07471054147508643, 0.08775344251508531, 0.04599969706309533, 0.10879798933968804, 0.02290124444838471, 0.012960693104005376, 0.021219507823901484, 0.012050471465158419, 0.008135998751005163, 0.012815713311474583, 0.03203799252479553, 0.003248842722383419, 0.00116420061682601, 0.002733098889794657, 0.006911136179452413, 0.002230075042256015, 0.001083476752781981, 0.0068513600728053225, 0.003721007737825836, 0.007463620351223677, 0.001447385808562753, 0.017755143577401596, 0.012884428175524928, 0.008202426517722489, 0.012701066720211544, 0.005421275073487947, 0.006199931574

In [ ]:
def sdf_query(mesh, query_points):

  mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

  # Create a scene and add the triangle mesh
  scene = o3d.t.geometry.RaycastingScene()
  _ = scene.add_triangles(mesh)  # we do not need the geometry ID for mesh
  
  sdf = scene.compute_signed_distance(query_points)
  # print(query_points.shape, occupancy.shape)
  # We can visualize a slice of the distance field directly with matplotlib
  # plt.imshow(occupancy.numpy()[:, :, 70])
  #print(np.shape(sdf.numpy()))
  return sdf.numpy()

In [ ]:
tempdata = pickle.load(open(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\NeuroImg_LVNET\cuttail_temp.pkl","rb"))
tempdata_tex = pickle.load(open(rf"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\NeuroImg_LVNET\cuttail_temp_L_tex.pkl","rb"))
print(tempdata_tex.keys())
lv_temp = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata['lv']))
tri = tempdata_tex['cuttail_lv']
tri0 = tri.copy()
tri0[:,[0,2]] = tri[:,[2,0]]  # swap 0 and 2 to match the original mesh
lv_temp_tex = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata_tex['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tri0))

mesh1 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata_tex['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata_tex['tri1']))
mesh2 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata_tex['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata_tex['tri2']))
mesh3 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata_tex['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata_tex['tri3']))
mesh4 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tempdata['hippo']))
lv_temp.paint_uniform_color([0.7, 0.7, 1.0])
lv_temp_tex.paint_uniform_color([0.7, 1.0, 1.0])
mesh1.paint_uniform_color([1.0, 0.7, 0.7])
mesh2.paint_uniform_color([0.7, 1.0, 0.7])
mesh3.paint_uniform_color([0.7, 0.7, 1.0])
o3d.visualization.draw_geometries([mesh1,mesh2,mesh3, mesh4], mesh_show_wireframe=True)



In [ ]:
verts = np.asarray(tempdata['cuttail_vert']).astype(np.float32)
print(verts.shape)
mesh1_sdf = np.abs(sdf_query(mesh1, verts))
mesh2_sdf = np.abs(sdf_query(mesh2, verts))
mesh3_sdf = np.abs(sdf_query(mesh3, verts))
mesh4_sdf = np.abs(sdf_query(mesh4, verts))
print(mesh1_sdf.shape, mesh1_sdf[:5])

# index of mesh1_sdf<0.01
index1 = np.where(mesh1_sdf<0.05)[0]
index2 = np.where(mesh2_sdf<0.05)[0]
index3 = np.where(mesh3_sdf<0.05)[0]
index4 = np.where(mesh4_sdf<0.05)[0]
# N,3 size data to N*3 size data

print(index4)
print(index4.shape)
print(index1.shape, index1[:5])
tri1, tri2, tri3, tri4 = [], [], [], []
for i in range(tempdata['lv'].shape[0]):

        if (tempdata['lv'][i,0] in index1) and (tempdata['lv'][i,1] in index1) and (tempdata['lv'][i,2] in index1):
            tri1.append(tempdata['lv'][i].tolist())
        elif (tempdata['lv'][i,0] in index2) and (tempdata['lv'][i,1] in index2) and (tempdata['lv'][i,2] in index2):
            tri2.append(tempdata['lv'][i].tolist())
        elif (tempdata['lv'][i,0] in index3) and (tempdata['lv'][i,1] in index3) and (tempdata['lv'][i,2] in index3):
            tri3.append(tempdata['lv'][i].tolist())
        elif (tempdata['lv'][i,0] in index4) and (tempdata['lv'][i,1] in index4) and (tempdata['lv'][i,2] in index4):
            tri4.append(tempdata['lv'][i].tolist())
tri1 = np.asarray((tri1))
print(tri1.shape)
tri2 = np.asarray((tri2))
print(tri2.shape)
tri3 = np.asarray((tri3))
print(tri3.shape)
tri4 = np.asarray((tri4))
print(tri4.shape)

mesh1 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tri1))
mesh2 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tri2))
mesh3 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tri3))
mesh4 = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(tempdata['cuttail_vert']), triangles=o3d.utility.Vector3iVector(tri4))
mesh1.paint_uniform_color([1.0, 0.7, 0.7])
mesh2.paint_uniform_color([0.7, 1.0, 0.7])
mesh3.paint_uniform_color([0.7, 0.7, 1.0])
mesh4.paint_uniform_color([0.7, 0.85, 0.8])
        
o3d.visualization.draw_geometries([mesh1,mesh2,mesh3,mesh4], mesh_show_wireframe=True)



In [ ]:
print(tempdata_tex['cuttail_lv'].max())
print(tempdata_tex['cuttail_hippo'].min())

In [ ]:
p_all = []


for index in [index1, index2]: # caudate, thalamus, opposite LV
    
    p_all.append(p_value[index])
p_all.append(p_value)
plt.boxplot(
    p_all,
    vert=True,              # vertical boxes (set False for horizontal)
    labels=['Caudate', 'Thalamus', 'left LV'],
    showmeans=True          # draws a green triangle at the mean
)

plt.ylabel("Value")
plt.title("Boxplot of 5 data sets")
plt.grid(axis="y", linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()



In [ ]:
L = o3d.io.read_triangle_mesh(rf"age_mannwhitneyu_L.obj")
R = o3d.io.read_triangle_mesh(rf"age_mannwhitneyu_R.obj")
L_color = np.load("age_p_L.npy")
R_color = np.load("age_p_R.npy")

print(L_color.shape, R_color.shape)

L_vert = np.asarray(L.vertices)
L_vert[:,0] = -L_vert[:,0]
L_vert[:,0] += 2.5
L_tri = np.asarray(L.triangles)
L_tri0 = L_tri.copy()
L_tri0[:,[0,2]] = L_tri[:,[2,0]]

L.vertices = o3d.utility.Vector3dVector(L_vert)
L.triangles = o3d.utility.Vector3iVector(L_tri0)


coord_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20, origin=[25, 0,  0])
o3d.visualization.draw_geometries([L, R], mesh_show_wireframe=True)

(2826,) (2826,)


: 